# **INSY-5336 PYTHON FINAL PROJECT**
**Submitted By: Sanjivni Rana (1001820277)**





The code below uses bs4: BeautifulSoup library for web scraping. We have taken the url of IMDB's Top 250 Movies, specified headers and page which are important inputs for the Beautiful Soup library. We then create an object soup and find the movie details using its functions find(), find_all(), get(), etc.
From the url we scrape the links of all Top 250 movies using the soup object and its functions and store it in a list called movie_links.



In [ ]:
#importing all necessary libraries for web scraping
import requests
from bs4 import BeautifulSoup
import urllib.request, urllib.parse, urllib.error

#defining all necesary parameters to pass to beautiful soup's object
url = 'https://www.imdb.com/chart/top/?ref_=nv_mv_250'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}
page = requests.get(url, headers = headers)
soup = BeautifulSoup(page.content, "html.parser")
movies = soup.find_all('td', class_ = 'titleColumn')

#creating a list to store all the movie links for further information scraping
movie_links = []
for i in movies:
    hyperlink = i.find('a')
    url = hyperlink.get('href')
    url = 'https://www.imdb.com' +url
    movie_links.append(url)

Then we iterate through each movie link in the movie_links list and for each movie we find its title (using find h1) and similarly other information like director'name, bio, dob, place of birth and movie genres.
As seen in the code, sepearate dictionaries, sub dictionaries are instantiated to store the information scraped. dir_subDict stores directors bio, dob and place as key-value pair which is then stores as the value into dirDict with director's name as the key.
Similarly genres list stores all the genres of a movie. sub_dic stores the director's name as the key and genres list as the value which is then stored as a value in movie_dic with movie's name as the key.
All the data is scraped using for-loops and soup object and its functions and stored in the dict and sub-dict. Finally at the end of each iteration movie_dic is appended to the all_movies list. This list is printed at the end to see all 250 movie's titles and their director and genres.


In [ ]:
#using regex to clean data and time to make the program take breaks
import re
import time

try:
  #dirDict dictionary to store director's bio, dob and dop (for bonus question)
  #all_movies list stores the top 250 movies, their director and genres
  dirDict = {}
  all_movies = []
  for movie_link in movie_links:

      #the object soup will be used to find the required data from the parsed html of the website
      page = requests.get(movie_link, headers = headers)
      soup = BeautifulSoup(page.content, "html.parser")

      title_row = soup.find('h1')
      title = title_row.get_text()

      #genres list will store all the genres of a movie
      #sub_dic will store the director's name as key and genres list as the value
      #movie_dic will store the movie name as key and sub_dic as the value
      genres = []
      sub_dic = {}
      movie_dic = {}

      #this block of code scrapes the information of genre
      #for the genre with differen class, a separate if case is used
      genre_row = soup.find('div', attrs = {'class' : 'ipc-chip-list sc-16ede01-4 bMBIRz', 'data-testid' : 'genres'})
      if(genre_row == None):
        genre_row = soup.find('div', attrs = {'class' : 'ipc-chip-list sc-16ede01-5 ggbGKe', 'data-testid' : 'genres'})
      genre_tags = genre_row.find_all('ul')

      #the ul tags for all genres stored in the list are iterated one by one
      #the actual genre name is extracted using get_text() and appended to genres list
      for i in genre_tags:
          genre = i.get_text()
          genres.append(genre)

      #this block of code scrapes the directors information from the html
      director_block = soup.find('li', attrs = {'class' : 'ipc-metadata-list__item', 'data-testid' : 'title-pc-principal-credit'})
      dir_list = director_block.find_all('a', class_ = 'ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')

      #for the movies with multiple directors, we iterate through each directors anchor tags in dir_list
      #the genres list populated above is stores as a value i sub_dict with key as director's name
      if len(dir_list) != 0:
        for d in dir_list:
          if d:
            sub_dic[d.get_text()] = genres

            #this is a part of the bonus question
            #each distinct director's info is stored in dir_dict
            #the link for director's bio page is scraped and information if extracted
            if d.get_text() not in dirDict:
              bioLink = d.get('href')
              bioLink = 'https://www.imdb.com' +bioLink
              dirPage = requests.get(bioLink, headers = headers)
              dirSoup = BeautifulSoup(dirPage.content, "html.parser")

              seeMoreLink = dirSoup.find('span', class_='see-more inline nobr-only')

              if seeMoreLink:
                seeMoreLink = seeMoreLink.find('a')
                fullBioLink = seeMoreLink.get('href')
                fullBioLink = 'https://www.imdb.com' +fullBioLink
                dirBioPage = requests.get(fullBioLink, headers = headers)
                dirBioSoup = BeautifulSoup(dirBioPage.content, "html.parser")

                #the dir_subDict stores director's bio, dob and dop
                #dir_subDict is passed as a value to the dirDict with director's name as the key
                dir_subDict = {}

                bio = dirBioSoup.find('div', class_='soda odd')
                if bio:
                  bioStr = bio.get_text()
                  bioStr = bioStr.strip()
                  bioStr = re.sub('\n', '', bioStr)
                  dir_subDict['bio'] = bioStr

                born = dirBioSoup.find('tr', class_='even')
                if born:
                  dob = born.find_all('a')
                  if len(dob) == 3:
                    dir_subDict['dob'] = dob[0].get_text()+ ', ' +dob[1].get_text()
                    dir_subDict['place'] = dob[2].get_text()
                  elif len(dob) == 2:
                    dir_subDict['dob'] = dob[0].get_text()
                    dir_subDict['place'] = dob[1].get_text()
                  elif len(dob) == 1:
                    dir_subDict['dob'] = dob[0].get_text()
                  
                if len(dir_subDict) != 0:
                  dirDict[d.get_text()] = dir_subDict

      movie_dic[title] = sub_dic
      all_movies.append(movie_dic)
      time.sleep(1)

  print(all_movies)

except ValueError:
    print('Something went wrong! Please try again.')

[{'The Shawshank Redemption': {'Frank Darabont': ['Drama']}}, {'The Godfather': {'Francis Ford Coppola': ['Crime', 'Drama']}}, {'The Dark Knight': {'Christopher Nolan': ['Action', 'Crime', 'Drama']}}, {'The Godfather: Part II': {'Francis Ford Coppola': ['Crime', 'Drama']}}, {'12 Angry Men': {'Sidney Lumet': ['Crime', 'Drama']}}, {"Schindler's List": {'Steven Spielberg': ['Biography', 'Drama', 'History']}}, {'The Lord of the Rings: The Return of the King': {'Peter Jackson': ['Action', 'Adventure', 'Drama']}}, {'Pulp Fiction': {'Quentin Tarantino': ['Crime', 'Drama']}}, {'The Lord of the Rings: The Fellowship of the Ring': {'Peter Jackson': ['Action', 'Adventure', 'Drama']}}, {'The Good, the Bad and the Ugly': {'Sergio Leone': ['Adventure', 'Western']}}, {'Forrest Gump': {'Robert Zemeckis': ['Drama', 'Romance']}}, {'Fight Club': {'David Fincher': ['Drama']}}, {'Inception': {'Christopher Nolan': ['Action', 'Adventure', 'Sci-Fi']}}, {'The Lord of the Rings: The Two Towers': {'Peter Jackson

The code below stores all the data from all_movies list into a CSV named 'SanjivniRana_movies.csv'. For this csv library is imported and used. The object of csv.writer writes each row into the CSV. This is done by iterating through each element of all_movies list using for-loop and then further iterating the dict and sub dict to append values in the right manner. At the end of each loop, writer.writerow appends each movie with its director and genre into each row in the corresponsing order.

In [ ]:
#this block of code stores the list of movies dictionary into a csv
#for loop is used to iterate over the list, dictionary and the sub-dictionary
#first row is the title and then values are inserted
import csv

try:
  with open('SanjivniRana_movies.csv', 'w') as f:
      writer = csv.writer(f,)
      writer.writerow(['S.No.','Movie Name','Director','Genre'])
      count = 1
      for movie in all_movies:
        col1 = ''
        for movieKey,subDict in movie.items():
          col1 = movieKey
          col2 = ''
          for dirKey,genreVal in subDict.items():
            col2 = col2 + dirKey + ', '
            col3 = ''
            if col3 == '':
              for gen in list(genreVal):
                if col3 == '':
                  col3 = gen
                else:
                  col3 = col3 + ', ' + gen 
          col2 = col2[0:-2]
        writer.writerow([count,col1,col2,col3])
        count = count+1

except ValueError:
    print('Something went wrong! Please try again.')

The code below contains three functions handleUpperLowerCase(), fetchMovieDetails() and fetchDirMoviesAndGenreCount(). The first function simply handles lower and upper casing of the input. The second function takes movie name as the parameter which is input by the user. Just like before, for-loop is used to iterate over each dictionary in the list and each sub dictionary in each dictionary to match the movie name and extract the required information. The method used to iterate over a sub-dict is for movieKey,subDict in movie.items(). The third function works in the similar way but takes director name as the parameter instead.


In [ ]:
#this block contains functions which are used in the code below
#kindly run this before running the main code

#code to handle upper and lower casing
def handleUpperLowerCase(string):

  try:
    lowerStr = ''
    for ch in string:
      if ch.isalpha():
        ch = ch.lower()
        lowerStr = lowerStr + ch
      else:
        lowerStr = lowerStr + ch
    return lowerStr
    
  except ValueError:
    print('Something went wrong! Please try again.')

#this function is used for option 1 which finds movie's information
#nested for-loop is used to iterate over the list, dictionary and the sub-dictionary
#if-else is used to for various null-checks
#for-loop iterates over each dictionary in the list and each subdictionary in the dictionary to match the input provided by the user
#if the string matches then the required information is extracted into a new variable and displayed to the user
def fetchMovieDetails(movieName):

  try:
    movieDir = ''
    movieGenres = ''
    for movie in all_movies:
      for movieKey,subDict in movie.items():
        movieKey = handleUpperLowerCase(movieKey)
        if movieKey == movieName:
          for dirKey,genreVal in subDict.items():
            if movieDir == '':
                movieDir = dirKey
            else:
                movieDir = movieDir + ', ' + dirKey
            if(movieGenres == ''):
              for gen in list(genreVal):
                if movieGenres == '':
                  movieGenres = gen
                else:
                  movieGenres = movieGenres + ', ' + gen
    print('  ')
    if movieDir and movieGenres:
      print('The director of the movie is',movieDir)
      print('The genre of the movie is',movieGenres)
    else:
      print('The information about',movieName,'is not in our data. Please try again.')

  except ValueError:
    print('Something went wrong! Please try again.')

#this function is used for option 2 and 3 which finds director's information stores in sub-dictionary
#nested for-loop is used to iterate over the list, dictionary and the sub-dictionary
#if-else is used to for various null-checks
#for-loop iterates over each dictionary in the list and each subdictionary in the dictionary to match the input provided by the user
#if the string matches then the required information is extracted into a new variable and displayed to the user
def fetchDirMoviesAndGenreCount(director):

  try:
    dir = ''
    dirMovies = ''
    dirGenCount = ''
    dirGenreDict = dict()
    for movie in all_movies:
      for movieKey,subDict in movie.items():
        for dirKey,genreVal in subDict.items():
          if dirKey.lower() == director.lower():
            dir = dirKey;
            if dirMovies == '':
                dirMovies = movieKey
            else:
                dirMovies = dirMovies + ', ' + movieKey

            for gen in list(genreVal):
              dirGenreDict[gen]= dirGenreDict.get(gen, 0) + 1

            dirGenCount = 'His most directed genres are '
            for gen,count in dirGenreDict.items():
              dirGenCount = dirGenCount + gen + ':' + str(count) + ', '
            dirGenCount = dirGenCount[0:-2] + '.'

    print(' ')
    if len(dirGenreDict) != 0:
      print(dir,'has directed',dirMovies)
      print(dirGenCount)
    else:
      print('The information about',director,'is not in our data. Please try again.')
    return dirGenreDict

  except ValueError:
    print('Something went wrong! Please try again.')

The code below checks the option chosen by the user. We have used if condition for this. It then enters one of the if blocks and calls the function specified above to extract the required information from all_movies list.
For the third option, we call the function to get the genreDict and then form genre count vector from it. This is done through for-loop iteration and if-else checking. Each genre key is matched in both the dictionaries and their corresponding count is set in the vector, if not present then the vector value is assigned as 0. This vector is then used to calculate the cosine similarity.

In [ ]:
#math library is imported to use the square root function to calculate cosine similarity
import math

option = input('What do you want to check on IMDB? (Please choose ‘movie’, ‘director’, or ‘comparison’): ')

try:
  option = option.lower()

  #each if block is specified for each option chosen by the user
  #the input is sent to the functions declared above to find the information through nested loops
  if option == 'movie':
    movie = input('What movie do you want to check?: ')
    movieName = handleUpperLowerCase(movie)
    fetchMovieDetails(movieName)

  elif option == 'director':
    director = input('Who do you want to check?: ')
    fetchDirMoviesAndGenreCount(director)

  elif option == 'comparision':
    print('Who do you want to compare?')
    print('  ')
    dir1 = input('input 1: ')
    dir2 = input('input 2: ')

    dir1genDict = dict()
    dir2genDict = dict()
    dir1genDict = fetchDirMoviesAndGenreCount(dir1)
    dir2genDict = fetchDirMoviesAndGenreCount(dir2)

    #this block of code forms the vectors of two chosen directors
    #once genre dictionary is passed from the above function, vector is created using the count
    #this is done using for-loop iteration in each dictionary and checking the key-value content
    if len(dir1genDict) != 0 and len(dir2genDict) != 0:

      dir1genVector = tuple()
      dir2genVector = tuple()

      for gen in dir1genDict:
        if gen in dir2genDict:
          dir1genVector = dir1genVector + (dir1genDict[gen],)
          dir2genVector = dir2genVector + (dir2genDict[gen],)
        else:
          dir1genVector = dir1genVector + (dir1genDict[gen],)
          dir2genVector = dir2genVector + (0,)

      for gen in dir2genDict:
        if gen not in dir1genDict:
          dir2genVector = dir2genVector + (dir2genDict[gen],)
          dir1genVector = dir1genVector + (0,)
        
      cosine = 0
      lengthX = 0
      lengthY = 0
      vectorProduct = 0

      #this block of code calculates cosine similarity
      #it takes two vectors created above and calculates using for-loop iteration
      for i in range(len(dir1genVector)):
        vectorProduct = vectorProduct + (dir1genVector[i]*dir2genVector[i])
        lengthX = lengthX + (dir1genVector[i]**2)
        lengthY = lengthY + (dir2genVector[i]**2)
      lengthX = math.sqrt(lengthX)
      lengthY = math.sqrt(lengthY)

      if lengthX*lengthY != 0:
        cosine = vectorProduct/(lengthX*lengthY)

      print(' ')
      print('Based on that, they have a cosine similarity score of',round(cosine,2))

    else:
      print(' ')
      print('Since the director is not in our data. Cosine cannot be calculated.')

  else:
    print(' ')
    print('Please input a valid choice!')

except ValueError:
  print('Something went wrong! Please try again.')

What do you want to check on IMDB? (Please choose ‘movie’, ‘director’, or ‘comparison’): movie
What movie do you want to check?: Avengers: Infinity WAR
  
The director of the movie is Anthony Russo, Joe Russo
The genre of the movie is Action, Adventure, Sci-Fi


# **BONUS QUESTION**

The code below executes the bonus part of the project, i.e. display selected movie's directors bio, dob and place of birth. As specified above we have stored this information in a dictionary of dictionary named dirDict while scraping and storing information in all_movies. The structure of dirDict is : {DirectorName: {Bio:Value},{DOB:Value},{Place:Value}}. The movie name input by the user is searched in the all_movies list of dictionaries, if found then its director's name is searched in the dirDict dictionary and the details are displayed to the user.

In [ ]:
#this block of code displays the chosen director's information
#nested for-loop is used to iterate over the list, dictionary and the sub-dictionary
#if-else is used to for various null-checks
#each of the information i.e. bio, dob and place are stored as key-values in the subdictionary with director name as the key
#for-loop iterates over each dictionary in the list and each subdictionary in the dictionary to match the input provided by the user
#if the string matches then the required information is extracted into a new variable and displayed to the user
movieName = input('Which movie do you wish to choose?: ')
option = input('What do you wish to see about this movie?: ')

try:
  movieName = handleUpperLowerCase(movieName)

  isPresent = 0
  if option.lower() == 'director':
    for movie in all_movies:
      for movieKey,subDict in movie.items():
        movieKeyLower = handleUpperLowerCase(movieKey)
        if movieKeyLower == movieName:
          isPresent = 1
          for dirKey,genreVal in subDict.items():
            print(' ')
            print('The director of',movieKey,'is',dirKey)
            if dirKey in dirDict:
              if len(dirDict[dirKey]) == 3 or len(dirDict[dirKey]) == 2:
                bio = list(dirDict[dirKey].values())[0]
                dob = list(dirDict[dirKey].values())[1]
                place = list(dirDict[dirKey].values())[2]
                print(dirKey,'is born on',dob)
                print(dirKey,'is born in',place)
                print(bio)
              if len(dirDict[dirKey]) == 1:
                print(dirKey,'is born on',dob)
                print('The directors place of birth is not specified on the site.')
                print(bio)

    if isPresent == 0:
      print(' ')
      print('The information on',movieName,'is not in our data. Please try again.')

  else:
    print(' ')
    print('Please input a valid choice!')

except ValueError:
  print('Something went wrong! Please try again.')

Which movie do you wish to choose?: FINDING nemo
What do you wish to see about this movie?: director
 
The director of Finding Nemo is Andrew Stanton
Andrew Stanton is born on December 3, 1965
Andrew Stanton is born in Boston, Massachusetts, USA
Andrew Stanton has been a major creative force at Pixar Animation Studios since 1990, when he became the second animator and ninth employee to join the company's elite group of computer animation pioneers. As Vice President, Creative he currently oversees all shorts and feature projects at the studio. Stanton wrote and directed the Academy Award®-winning Disney and Pixar feature film "WALL.E," for which he also received a Best Original Screenplay Oscar®-nomination. In 2016 Stanton directed Disney and Pixar's "Finding Dory," which, upon release, became the highest-grossing domestic animated feature of all time and in 2019 Stanton served as screenwriter and executive producer of "Toy Story 4."Stanton made his directorial debut with the record-sha